In [ ]:
import os
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import json

In [ ]:
signals_to_consider = ['acceleration_x', 'acceleration_y', 'acceleration_z', 'accelerator_pedal', 'accelerator_pedal_gradient_sign',
                 'angular_velocity_omega_x', 'angular_velocity_omega_y', 'angular_velocity_omega_z', 'brake_pressure',
                 'distance_pulse_front_left', 'distance_pulse_front_right', 'distance_pulse_rear_left', 'distance_pulse_rear_right',
                 'latitude_degree', 'latitude_direction', 'longitude_degree', 'longitude_direction',
                 'pitch_angle', 'roll_angle', 'steering_angle_calculated', 'steering_angle_calculated_sign', 'vehicle_speed']

In [ ]:
def get_data(path):
    
    with open(path) as f:
       data = json.load(f)

    return data   
    

def extract_signal(json_file, signal_name):
    return pd.DataFrame(json_file[signal_name]['values'], columns=['timestamp',signal_name])

def get_directory_for_dataset():
    element_to_remove = '.DS_Store'

    #directory = os.listdir('datasets/Audi a2d2/camera_lidar')
    directory =['20190401_121727','20180810_150607','20190401_145936']
    
    if element_to_remove in directory:
        directory.remove(element_to_remove)
     
    directory_file_name_prefix = [item.replace("_", "") for item in directory]
    return directory, directory_file_name_prefix

def clean_resample_and_store(data, signals_names, directory_for_saving_data):#, resample_frequence_in_sec = 1):
    #extracted_data
    for i in signals_names:
        if i == 'acceleration_x':
            print(data)
            extracted_data = extract_signal(data, i)
        else:
            extracted_data = pd.merge(extracted_data, extract_signal(data, i),on='timestamp',how='outer',sort=True)

    # Convert microseconds to seconds
    extracted_data['timestamp'] = extracted_data['timestamp'] / 1e6
    extracted_data['timestamp'] = pd.to_datetime(extracted_data['timestamp'], unit='s')     
    extracted_data = extracted_data.set_index('timestamp')
    # resampling data
    extracted_data = extracted_data.resample('s').mean().ffill() 
    extracted_data.to_csv(directory_for_saving_data,
                          index=True)

In [ ]:
directory, directory_file_name_prefix = get_directory_for_dataset()

for i in tqdm(range(len(directory))):
    path_of_file_inside_subdir = directory_file_name_prefix[i] + '_bus_signals.json'
    file_path = 'datasets/Audi a2d2/camera_lidar/{}/bus/{}'.format(directory[i],path_of_file_inside_subdir)
    print(file_path)
    clean_resample_and_store(get_data(file_path),
                              signals_to_consider, 
                              directory_for_saving_data='datasets/resampled_{}.csv'.format(directory_file_name_prefix[i]))